In [1]:
import numpy as np
import pandas as pd
import pickle as pkl
from concurrent.futures import ThreadPoolExecutor
import os
from data_generation import generate_gaussian_noises_dict, generate_sparse_response, generate_perturbed_response


In [2]:
# TODO: Hyperparameters search for omp

##* Data Generation
##* N (columns of dictionary) = 10k
##* d (dimension of signal) = 300, 600, 900, 1200, 1500, 2000
##* X_{ij} \sim N(0, \frac{1}{n}) (The same as N(0,1) them normalize)
##* m (sparse level) = 20, 40, 80

##* Noise level = 0.01, 0.05, 0.1, 0.2, 0.5

N = 10000
d = 1200
m = 40
noise_level_lst = [0, 0.01, 0.05, 0.1, 0.2, 0.5]
trial_num = 20

cv_num = 10

In [3]:
##! Task 1: Given signal, dictionary, sparsity level and noise level, use testset (10% of the whole signal) to find the best K(depth) for omp

from data_generation import GaussianDataGenerator

seed = 0
Data_Geneartor = GaussianDataGenerator(N, d, m, 0.05, seed)

true_signal, dictionary, true_indices, true_coefficients, perturbed_signal = Data_Geneartor.shuffle()

In [4]:
def cv_split(true_signal, dictionary, cv_num):
    true_signal = true_signal.ravel()
    # true_signal is (1200, 1) and dictionary is (1200, 10000), cv both signal and dictionary by rows
    cv_signal = np.split(true_signal, cv_num)
    cv_dictionary = np.split(dictionary, cv_num)
    # Get the list of train and test set
    cv_res = []
    for i in range(cv_num):
        train_signal = np.concatenate(cv_signal[:i] + cv_signal[i + 1:], axis = 0)
        train_dictionary = np.concatenate(cv_dictionary[:i] + cv_dictionary[i + 1:], axis=0)
        test_signal = cv_signal[i]
        test_dictionary = cv_dictionary[i]
        cv_res.append((train_signal, train_dictionary, test_signal, test_dictionary))
    return cv_res

In [5]:
# We need a OMP class to do the cross validation and it need a fit and score function

def cal_cv_error(algorithm, cv_num, signal, dictionary):
    error_lst = []
    cv_res = cv_split(signal, dictionary, cv_num)
    for i in range(cv_num):
        train_signal, train_dictionary, test_signal, test_dictionary = cv_res[i]
        algorithm.fit(train_signal, train_dictionary)
        error_lst.append(algorithm.score(test_signal, test_dictionary))
    return np.mean(error_lst)

In [7]:
from algorithms import OMP

cal_cv_error(OMP(10), 10, true_signal, dictionary)

Singular matrix encountered in OMP
Singular matrix encountered in OMP
Singular matrix encountered in OMP
Singular matrix encountered in OMP
Singular matrix encountered in OMP
Singular matrix encountered in OMP
Singular matrix encountered in OMP
Singular matrix encountered in OMP
Singular matrix encountered in OMP


0.027813194807933837

In [ ]:
##! Task 2: Relationship between best K and noise level (std)



##! Task 3: Relationship between Test Error and noise level (std)